In [1]:
import pandas as pd

In [3]:
base = pd.read_pickle('../data/base_df.pkl')
snli = pd.read_pickle('../data/english_snli_generate_class_shot.pkl')
tweet = pd.read_pickle('../data/english_tweet_eval_generate_class_shot.pkl')
sarcasmania = pd.read_pickle('../data/sarcasmania_generate_class_shot.pkl')
word_context = pd.read_pickle('../data/word_context_generate_class_shot.pkl')

snli['correctness'] = snli['ratio_'].map(lambda x: abs(1 - x))
tweet['correctness'] = tweet['ratio_'].map(lambda x: abs(1 - x))
sarcasmania['correctness'] = sarcasmania['ratio_'].map(lambda x: abs(1 - x))
word_context['correctness'] = word_context['ratio_'].map(lambda x: abs(1 - x))
base['correctness'] = base['ratio'].map(lambda x: abs(1 - x))

In [4]:
# change values in order_ column with {'AB': 'Maj-Min', 'BA': 'Min-Maj'} in sarcasmania
sarcasmania['order_'] = sarcasmania['order_'].map({'AB': 'Maj-Min', 'BA': 'Min-Maj'})
word_context['order_'] = word_context['order_'].map({'AB': 'Maj-Min', 'BA': 'Min-Maj'})
snli['order_'] = snli['order_'].map({'ABC': 'Maj-Min', 'CBA': 'Min-Maj'})
tweet['order_'] = tweet['order_'].map({'ADBC': 'Maj-Min', 'CBDA': 'Min-Maj'})

In [5]:
def o_p_analysis(df, name):
    df = df[df['correctness'] == 1][df['fewshot_'] > 1][df['permutation_'] != 'None'].groupby(['model_','permutation_','order_']).mean()['mean_f1_mean'].unstack().unstack().dropna(axis=1)
    df.columns = ["_".join(col).strip() for col in df.columns.to_flat_index()]
    # df.index = [i+f'_{name}' for i in df.index]
    df['dataset'] = name
    df['model'] = df.index
    df = df.set_index(['dataset', 'model'])
    # rename column that contains the word "RANDOM" to "RANDOM"
    df.columns = [i if 'RANDOM' not in i else 'RANDOM' for i in df.columns]
    # make the RANDOM column the last column
    df = df[[col for col in df.columns if col != 'RANDOM'] + ['RANDOM']]
    return df

In [6]:
dfs = []
for i, name in zip([snli, tweet, sarcasmania, word_context], ['snli', 'tweet', 'sarcasmania', 'word_context']):
    dfs.append(o_p_analysis(i, name))
all_o_p = pd.concat(dfs)

/tmp/ipykernel_12840/1340402950.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['correctness'] == 1][df['fewshot_'] > 1][df['permutation_'] != 'None'].groupby(['model_','permutation_','order_']).mean()['mean_f1_mean'].unstack().unstack().dropna(axis=1)
/tmp/ipykernel_12840/1340402950.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['correctness'] == 1][df['fewshot_'] > 1][df['permutation_'] != 'None'].groupby(['model_','permutation_','order_']).mean()['mean_f1_mean'].unstack().unstack().dropna(axis=1)
/tmp/ipykernel_12840/1340402950.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['correctness'] == 1][df['fewshot_'] > 1][df['permutation_'] != 'None'].groupby(['model_','permutation_','order_']).mean()['mean_f1_mean'].unstack().unstack().dropna(axis=1)
/tmp/ipykernel_12840/1340402950.py:2: UserWarning: Boolean Series key will be reindexed to match Dat

In [8]:
all_o_p

Maj-Min_AAA  Maj-Min_ABC  \
dataset      model                                                            
snli         meta-llama/Llama-3.1-8B-Instruct         0.749992     0.770067   
             meta-llama/Llama-3.3-70B-Instruct        0.821106     0.820398   
             mistralai/Mixtral-8x7B-Instruct-v0.1     0.814770     0.801989   
tweet        meta-llama/Llama-3.1-8B-Instruct         0.772610     0.778044   
             meta-llama/Llama-3.3-70B-Instruct        0.766688     0.775284   
             mistralai/Mixtral-8x7B-Instruct-v0.1     0.728177     0.739550   
sarcasmania  meta-llama/Llama-3.1-8B-Instruct         0.720517     0.777615   
             meta-llama/Llama-3.3-70B-Instruct        0.821567     0.847246   
             mistralai/Mixtral-8x7B-Instruct-v0.1     0.945242     0.945275   
word_context meta-llama/Llama-3.1-8B-Instruct         0.613379     0.637683   
             meta-llama/Llama-3.3-70B-Instruct        0.725476     0.727791   
             mistralai/Mixtral-8x7B-Instruct-v0.1     0.684138     0.678926   

                                                   Min-Maj_AAA  Min-Maj_ABC  \
dataset      model                                                            
snli         meta-llama/Llama-3.1-8B-Instruct         0.739920     0.760294   
             meta-llama/Llama-3.3-70B-Instruct        0.793936     0.805052   
             mistralai/Mixtral-8x7B-Instruct-v0.1     0.778382     0.790896   
tweet        meta-llama/Llama-3.1-8B-Instruct         0.781428     0.774350   
             meta-llama/Llama-3.3-70B-Instruct        0.772015     0.771119   
             mistralai/Mixtral-8x7B-Instruct-v0.1     0.738244     0.731447   
sarcasmania  meta-llama/Llama-3.1-8B-Instruct         0.708722     0.705805   
             meta-llama/Llama-3.3-70B-Instruct        0.847944     0.839643   
             mistralai/Mixtral-8x7B-Instruct-v0.1     0.911844     0.910294   
word_context meta-llama/Llama-3.1-8B-Instruct         0.650864     0.621419   
             meta-llama/Llama-3.3-70B-Instruct        0.718476     0.721577   
             mistralai/Mixtral-8x7B-Instruct-v0.1     0.684455     0.692049   

                                                     RANDOM  
dataset      model                                           
snli         meta-llama/Llama-3.1-8B-Instruct      0.764243  
             meta-llama/Llama-3.3-70B-Instruct     0.810760  
             mistralai/Mixtral-8x7B-Instruct-v0.1  0.798897  
tweet        meta-llama/Llama-3.1-8B-Instruct      0.777489  
             meta-llama/Llama-3.3-70B-Instruct     0.773760  
             mistralai/Mixtral-8x7B-Instruct-v0.1  0.734490  
sarcasmania  meta-llama/Llama-3.1-8B-Instruct      0.732299  
             meta-llama/Llama-3.3-70B-Instruct     0.838994  
             mistralai/Mixtral-8x7B-Instruct-v0.1  0.927578  
word_context meta-llama/Llama-3.1-8B-Instruct      0.631046  
             meta-llama/Llama-3.3-70B-Instruct     0.720560  
             mistralai/Mixtral-8x7B-Instruct-v0.1  0.684805

In [9]:
all_o_p.mean()

Maj-Min_AAA    0.763639
Maj-Min_ABC    0.774989
Min-Maj_AAA    0.760519
Min-Maj_ABC    0.760329
RANDOM         0.766243
dtype: float64

In [10]:
llama_small = all_o_p[all_o_p.index.get_level_values(1) == 'meta-llama/Llama-3.1-8B-Instruct'].mean()

In [11]:
llama_big = all_o_p[all_o_p.index.get_level_values(1) == 'meta-llama/Llama-3.3-70B-Instruct'].mean()

In [12]:
mix = all_o_p[all_o_p.index.get_level_values(1) == 'mistralai/Mixtral-8x7B-Instruct-v0.1'].mean()

In [13]:
df = pd.concat([llama_small, llama_big, mix], axis=1)
df.columns = ['Llama-3.1-8B-Instruct', 'Llama-3.3-70B-Instruct', 'Mixtral-8x7B-Instruct-v0.1']
df.T

,Maj-Min_AAA,Maj-Min_ABC,Min-Maj_AAA,Min-Maj_ABC,RANDOM
Llama-3.1-8B-Instruct,0.714125,0.740852,0.720233,0.715467,0.726269
Llama-3.3-70B-Instruct,0.783709,0.792680,0.783093,0.784348,0.786018
Mixtral-8x7B-Instruct-v0.1,0.793082,0.791435,0.778231,0.781171,0.786443
